In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat or 'tfidf' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
def simple_ens(model_name,k=3,rnd=233,lr=0.05,feature_fraction=0.9,bagging_fraction=0.9,
               bag_frec=3,met='binary_logloss',max_d=3):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cls_auc_res = [0,0,0,0,0,0]
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x

            # share params
            params = {
                    'application': 'binary',
                    #'num_leaves': 8,
                    #'lambda_l1': 1,
                    'lambda_l2': 1.0,
                    'max_depth': max_d,
                    'metric': met, # or auc
                    'data_random_seed': 2,
                    'learning_rate':lr,
                    # 'bagging_fraction': bagging_fraction,
                    # 'bagging_freq':bag_frec,
                    'feature_fraction': feature_fraction,

                    }
            if met == 'auc':
                s_round = 60
            else:
                s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i], lr, feature_fraction, max_d)
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        cls_auc_res[i] = val_auc_l
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred, cls_auc_res

print('done')

file path ../features/fasttext_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/fasttext_lstm_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn2d_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_5_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_5_feat.pkl
(159571, 6) (15

In [ ]:
lgb_res,tmp_auc_res = simple_ens('lgb',10,666,0.05,0.6)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# add 7 base fasttext NN models
# all loss avg 0.0319116484218 0.0358161833583 all auc avg 0.994546891137 0.991249510282 PUB 9866

# rm tfidf test
# all loss avg 0.0319555637279 0.0357756335619 all auc avg 0.994512718368 0.991251471241

# change to stratified
# all loss avg 0.0316412744167 0.0357232681944 all auc avg 0.994584962017 0.991293307537 pub 9866

# change to base model 5 fold, add word batch, tilli, lgb feat
# feat dim 217
# all loss avg 0.031983129767200906 0.035387924581 all auc avg 0.9945427088311004 0.99188514127 PUB 9870

# add more base models
# feat dim 247, all loss avg 0.031885221777487156 0.0353958  all auc avg 0.9945986685511962 0.9919487331094685

# change early stopping to 30, and test later part
# all loss avg 0.03208696729295824 0.035406264613808025 all auc avg 0.9945261773637045 0.991946174432887

# add fasttext lstm v1
# all loss avg 0.031977558955160815 0.03537909655655411 all auc avg 0.9945299073418443 0.9919307178575658

In [ ]:
# find best params for each column, early stopping = 30

best_pred = np.zeros((153164,6))
val_auc_res = [0,0,0,0,0,0]
for lr in [0.095,0.075,0.05]:
    for max_d in [3,4]:
        for s_rate in [0.4,0.5,0.6]:
            print('learning rate',lr,'max depth',max_d,'feature fraction',s_rate)
            lgb_res,tmp_auc_res = simple_ens('lgb',k=10,rnd=666,lr=lr,
                                 feature_fraction=s_rate,bagging_fraction=0.9,
                                 bag_frec=3,met='binary_logloss',max_d=max_d)
            # check for each cls
            for i in range(6):
                # find better params for this class
                if tmp_auc_res[i] > val_auc_res[i]:
                    val_auc_res[i] = tmp_auc_res[i]
                    best_pred[:,i] = lgb_res[:,i]
                    print('FIND BETTER PARAMS',lr,max_d,s_rate,list_classes[i])
            print('TEST PARAM DONE ------------------------------------------')

print(val_auc_res)
print(np.mean(val_auc_res))
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = best_pred
sample_submission.to_csv("../results/lgb_grid_search_fold10_stratified.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
                
            
# best auc params
# toxic 0.05,4,0.5
# severe toxic 0.075 3 0.5
# obs 0.075 3 0.6
# threat 0.095 3 0.5
# insult 0.075 0.5 4
# hate 0.05 0.5 3

learning rate 0.095 max depth 3 feature fraction 0.4
0 fold: ls 0.07055900792121056 0.07845851850483938 auc 0.9904317585515335 0.9869234386296798
1 fold: ls 0.07112099324841784 0.07132731780698526 auc 0.99019070161143 0.9898649322033591
2 fold: ls 0.06935924063822202 0.07556019794440967 auc 0.9908043936706946 0.9885162021559386
3 fold: ls 0.06497260357247332 0.07280091655739042 auc 0.992192585088142 0.9892247916632692
4 fold: ls 0.06894506079113848 0.07792721171645994 auc 0.9909000263031499 0.9878534906782339
5 fold: ls 0.06577492446228168 0.07645398182283704 auc 0.9919903108446425 0.9876221260056249
6 fold: ls 0.06797329110174438 0.07772442055333846 auc 0.9912493848473811 0.9876035860073693
7 fold: ls 0.07310551182282535 0.07122791366621857 auc 0.9894850349134301 0.9899879336591975
8 fold: ls 0.0704343227293608 0.07448200464055142 auc 0.9904142517680505 0.9892556209668459
9 fold: ls 0.06958413131063694 0.0776744481375048 auc 0.9906593620485566 0.9876591212710091
toxic 0.095 0.4 3
this

In [ ]:
def special_ens(model_name,k=3,rnd=233):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    params_list = [
        [0.05,4,0.5],
        [0.075,3,0.5],
        [0.075,3,0.6],
        [0.095,3,0.5],
        [0.075,4,0.5],
        [0.05,3,0.5],
    ]
    
    for i in range(6):
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        fold_cnt = 0
        
        # special params
        params = {
                'application': 'binary',
                #'num_leaves': 8,
                #'lambda_l1': 1,
                'lambda_l2': 1.0,
                'max_depth': params_list[i][1],
                'metric': 'binary_logloss', # or auc
                'data_random_seed': 2,
                'learning_rate':params_list[i][0],
                'feature_fraction': params_list[i][2],

                }
        print(params)
            
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test = test_x
            s_round = 30
            # train for each class
            d_train = lgb.Dataset(curr_x, curr_y[:,i])
            d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            try:
                train_pred = model.predict(curr_x)
                tmp_test_pred = model.predict(hold_out_x)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_auc_l = train_auc_l/k
        val_auc_l = val_auc_l/k
        print(list_classes[i])
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/6
        all_val_loss_l += val_loss_l/6
        all_train_auc_l += train_auc_l/6
        all_val_auc_l += val_auc_l/6
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

lgb_res = special_ens('lgb',10,666)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = lgb_res
sample_submission.to_csv("../results/lgb_log_csv_fold10_stratified_special.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# best params changed when base models changed
# all loss avg 0.03111512578966158 0.03534320053008906 all auc avg 0.9948524214184179 0.9918450388634261